In [1]:
import numpy as np
import pandas as pd
import pandas_datareader as  web
import matplotlib.pyplot as plt
import datetime
np.random.seed(7)


In [51]:
daily_drawdown_window = 10
daily_upside_window = 3

In [52]:
def calc_Daily_Drawdown_Daily_Upside(data, daily_drawdown_window, daily_upside_window):
    # We are going to use a trailing daily_drawdown_window trading day window
    # Calculate the max drawdown in the past window days for each day in the series.
    # Use min_periods=1 if you want to let the first daily_drawdown_window days data have an expanding window
    Roll_Max = data['Close'].rolling(daily_drawdown_window, min_periods=1).max() 
    Daily_Drawdown = 100*(data['Close']/Roll_Max - 1.0)

    # Next we calculate the minimum (negative) daily drawdown in that window.
    # Again, use min_periods=1 if you want to allow the expanding window
    Max_Daily_Drawdown = Daily_Drawdown.rolling(daily_drawdown_window, min_periods=1).min()
    
    # ---------------------------------------------------------------------------------------
    Roll_Min = data['Close'].rolling(daily_upside_window).min() 
    Daily_Upside = 100 * (data['Close']/Roll_Min - 1.0)
    Max_Daily_Upside = Daily_Upside.rolling(daily_upside_window, min_periods=1).max()
    
    # Modify Daily_Upside
    Daily_Upside = list(Daily_Upside)[daily_upside_window-1:] + list(Daily_Upside)[:daily_upside_window-1] 
    
    data['Daily_Drawdown'] = Daily_Drawdown
    data['Max_Daily_Drawdown'] = Max_Daily_Drawdown
    data['Daily_Upside'] = Daily_Upside 
#     data['Daily_Upside'] = data['Daily_Upside'] + 0.01

    data['Max_Daily_Upside'] = Max_Daily_Upside
    
    data['Pct_change_1'] = list(data['Close'].pct_change(1) * 100)[1:] + [np.nan]
    data['Pct_change_2'] = list(data['Close'].pct_change(2) * 100)[2:] + [np.nan, np.nan]
    
    return data.round(2)


In [53]:
data = pd.read_csv('../Data/Daily/US_Market/Indices/NDX_USD_20000101_to_20211231_daily_yahoo_finance.csv', parse_dates=['Date'])
data = data[['Date', 'Close']].set_index('Date')
data = data.loc["2021-01-01":]
data.tail()

,Close
Date,
2021-12-23,16308.209961
2021-12-27,16567.500000
2021-12-28,16488.660156
2021-12-29,16491.009766
2021-12-30,16429.099609


In [54]:
drawdown_name = f'Max drawdown in {daily_drawdown_window} days (%)' 
growth_name = 'Potential of growth (%)'
data = calc_Daily_Drawdown_Daily_Upside(data, daily_drawdown_window, daily_upside_window)
data[drawdown_name] = pd.cut(data['Daily_Drawdown'], bins=[-9, -8, -7, -6, -5, -4, -3, -2, -1, 0])
data[growth_name] = pd.cut(data['Daily_Upside'], bins=[0, 1, 2, 3, 4, 5, 6], right=False)
data['Next day change (%)'] = pd.cut(data['Pct_change_1'], bins=[-4, -3, -2, -1, 0, 1, 2, 3, 4], right=False)
data['Two days change (%)'] = pd.cut(data['Pct_change_2'], bins=[-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5, 6], right=False)


print('Min of Daily_Drawdown', data['Daily_Drawdown'].min())
print('Max of Daily_Upside', data['Daily_Upside'].max())

data.tail()

Min of Daily_Drawdown -8.22
Max of Daily_Upside 4.11


,Close,Daily_Drawdown,Max_Daily_Drawdown,Daily_Upside,Max_Daily_Upside,Pct_change_1,Pct_change_2,Max drawdown in 10 days (%),Potential of growth (%),Next day change (%),Two days change (%)
Date,,,,,,,,,,,
2021-12-23,16308.21,-0.15,-4.68,1.11,3.54,1.59,1.11,"(-1, 0]","[1.0, 2.0)","[1.0, 2.0)","[1.0, 2.0)"
2021-12-27,16567.50,0.00,-4.68,0.01,3.54,-0.48,-0.46,"(-1, 0]","[0.0, 1.0)","[-1.0, 0.0)","[-1.0, 0.0)"
2021-12-28,16488.66,-0.48,-4.68,0.00,2.39,0.01,-0.36,"(-1, 0]","[0.0, 1.0)","[0.0, 1.0)","[-1.0, 0.0)"
2021-12-29,16491.01,-0.46,-4.68,NaN,2.39,-0.38,NaN,"(-1, 0]",NaN,"[-1.0, 0.0)",NaN
2021-12-30,16429.10,-0.84,-4.68,NaN,1.11,NaN,NaN,"(-1, 0]",NaN,NaN,NaN


In [55]:
# data.loc['2015-08-10':'2015-08-27']

In [61]:
data.sort_values(by='Daily_Drawdown', ascending=True).head(5)

,Close,Daily_Drawdown,Max_Daily_Drawdown,Daily_Upside,Max_Daily_Upside,Pct_change_1,Pct_change_2,Max drawdown in 10 days (%),Potential of growth (%),Next day change (%),Two days change (%)
Date,,,,,,,,,,,
2021-03-04,12464.00,-8.22,-8.22,0.00,1.17,1.64,-1.32,"(-9, -8]","[0, 1)","[1.0, 2.0)","[-2, -1)"
2021-03-08,12299.08,-7.54,-8.22,3.68,1.64,4.03,3.68,"(-8, -7]","[3, 4)",NaN,"[3, 4)"
2021-02-25,12828.31,-7.09,-7.09,3.54,0.81,0.63,3.54,"(-8, -7]","[3, 4)","[0.0, 1.0)","[3, 4)"
2021-03-03,12683.33,-7.00,-7.09,1.64,3.54,-1.73,-0.12,"(-8, -7]","[1, 2)","[-2.0, -1.0)","[-1, 0)"
2021-05-12,13001.63,-6.93,-6.93,3.01,0.00,0.83,3.01,"(-7, -6]","[3, 4)","[0.0, 1.0)","[3, 4)"


In [57]:
df = pd.crosstab(data[drawdown_name], data[growth_name])
df = (df.div(df.sum(axis=1), axis=0)).round(2)
for col in df.columns:
    df[col] = df[col].astype(float).map("{:.0%}".format)
df

Potential of growth (%),"[0, 1)","[1, 2)","[2, 3)","[3, 4)","[4, 5)"
Max drawdown in 10 days (%),,,,,
"(-9, -8]",100%,0%,0%,0%,0%
"(-8, -7]",0%,33%,0%,67%,0%
"(-7, -6]",0%,50%,0%,50%,0%
"(-6, -5]",0%,33%,33%,33%,0%
"(-5, -4]",60%,20%,0%,7%,13%
"(-4, -3]",39%,33%,22%,6%,0%
"(-3, -2]",55%,23%,14%,9%,0%
"(-2, -1]",67%,20%,11%,2%,0%
"(-1, 0]",69%,26%,4%,1%,0%


In [58]:
df = pd.crosstab(data[drawdown_name], data['Next day change (%)'])
# df = (df.div(df.sum(axis=1), axis=0)).round(2)
# for col in df.columns:
#     df[col] = df[col].astype(float).map("{:.0%}".format)
df

Next day change (%),"[-4, -3)","[-3, -2)","[-2, -1)","[-1, 0)","[0, 1)","[1, 2)","[2, 3)","[3, 4)"
Max drawdown in 10 days (%),,,,,,,,
"(-9, -8]",0,0,0,0,0,1,0,0
"(-8, -7]",0,0,1,0,1,0,0,0
"(-7, -6]",0,0,0,0,1,0,1,0
"(-6, -5]",0,0,0,0,1,1,1,0
"(-5, -4]",0,3,0,4,5,0,3,0
"(-4, -3]",1,1,3,4,5,2,1,1
"(-3, -2]",0,4,1,4,5,7,1,0
"(-2, -1]",0,2,5,16,14,7,1,0
"(-1, 0]",1,2,10,45,60,21,2,0


In [59]:
df = pd.crosstab(data[drawdown_name], data['Two days change (%)'])
df = (df.div(df.sum(axis=1), axis=0)).round(2)
for col in df.columns:
    df[col] = df[col].astype(float).map("{:.0%}".format)
df

Two days change (%),"[-5, -4)","[-4, -3)","[-3, -2)","[-2, -1)","[-1, 0)","[0, 1)","[1, 2)","[2, 3)","[3, 4)","[4, 5)"
Max drawdown in 10 days (%),,,,,,,,,,
"(-9, -8]",0%,0%,0%,100%,0%,0%,0%,0%,0%,0%
"(-8, -7]",0%,0%,0%,0%,33%,0%,0%,0%,67%,0%
"(-7, -6]",0%,0%,0%,0%,0%,0%,50%,0%,50%,0%
"(-6, -5]",0%,0%,0%,0%,0%,0%,33%,33%,33%,0%
"(-5, -4]",7%,0%,13%,20%,7%,20%,20%,0%,7%,7%
"(-4, -3]",6%,0%,6%,6%,11%,17%,33%,17%,6%,0%
"(-3, -2]",0%,0%,9%,27%,14%,9%,23%,9%,9%,0%
"(-2, -1]",0%,9%,7%,9%,18%,36%,13%,7%,2%,0%
"(-1, 0]",0%,0%,6%,9%,25%,35%,21%,3%,1%,0%
